# 주식 및 ETF 데이터 수집

In [30]:
!pip install -U pykrx
# pip install/ beautifulsoup4 pandas openpyxl selenium webdriver-manager pykrx sqlalchemy pymysql


     ---------------------------------------- 0.0/60.9 kB ? eta -:--:--
     ---------------------------------------- 60.9/60.9 kB 3.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ----------- ---------------------------- 0.6/2.2 MB 13.3 MB/s eta 0:00:01
   ---------------------- ----------------- 1.2/2.2 MB 12.7 MB/s eta 0:00:01
   ------------------------------- -------- 1.7/2.2 MB 12.2 MB/s eta 0:00:01
   ---------------------------------------  2.2/2.2 MB 12.6 MB/s eta 0:00:01
   ---------------------------------------- 2.2/2.2 MB 10.7 MB/s eta 0:00:00
  Attempting uninstall: pykrx
    Found existing installation: pykrx 1.0.45
    Uninstalling pykrx-1.0.45:
      Successfully uninstalled pykrx-1.0.45



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
pip install configparser


Note: you may need to restart the kernel to use updated packages.


In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
from pykrx import stock
import datetime
import os
# from sqlalchemy import create_engine
import pymysql
import numpy as np
import configparser

import os

# ConfigParser 객체 생성
config = configparser.ConfigParser()

# config.ini 파일 경로 설정
config_file_path = 'E:\\AI\\pythonProject\\venv\\masicsplit\\config.ini'

# 경로가 올바른지 확인
if os.path.exists(config_file_path):
    print("config.ini 파일을 찾았습니다.")
    config.read(config_file_path)
else:
    print("config.ini 파일을 찾을 수 없습니다. 경로를 확인하세요.")

# mysql 섹션에서 설정값 가져오기
try:
    username = config['mysql']['user']
    password = config['mysql']['password']
    host = config['mysql']['host']
    database=config['mysql']['database']
    print(f"Username: {username}, Password: {password}, Host: {host}")
except KeyError as e:
    print(f"설정 파일에서 키를 찾을 수 없습니다: {e}")
except Exception as e:
    print(f"기타 오류 발생: {e}")




config.ini 파일을 찾았습니다.
Username: root, Password: @waren2ss, Host: 127.0.0.1


# 주식 및 ETF 데이터 수집

In [4]:
# 테이블 생성 함수
def create_tables():
    cur.execute('''
    CREATE TABLE IF NOT EXISTS stock_data (
        ticker VARCHAR(10),
        name VARCHAR(100),
        date DATE,
        open FLOAT,
        high FLOAT,
        low FLOAT,
        close FLOAT,
        volume BIGINT,
        value BIGINT,
        market_cap BIGINT,
        shares_outstanding BIGINT,
        PER FLOAT,
        PBR FLOAT,
        dividend FLOAT,
        BPS FLOAT,
        EPS FLOAT,
        DPS FLOAT,
        normalized_value FLOAT,
        PRIMARY KEY (ticker, date)
    )
    ''')
    cur.execute('''
    CREATE TABLE IF NOT EXISTS ticker_list (
        ticker VARCHAR(10) PRIMARY KEY,
        market VARCHAR(10),
        name VARCHAR(100),
        last_updated DATE,
        is_delisted BOOLEAN DEFAULT FALSE
    )
    ''')
    cur.execute('''
    CREATE TABLE IF NOT EXISTS ticker_status (
        ticker VARCHAR(10) PRIMARY KEY,
        status VARCHAR(20)
    )
    ''')
create_tables()

NameError: name 'cur' is not defined

In [2]:
# 모든 티커 리스트를 저장할 집합
all_tickers = set()

# 시작 날짜와 현재 날짜 설정
start_date = '19950102' # 이 시기부터 ticker list를 제공
end_date = datetime.datetime.now().strftime('%Y%m%d')

# 모든 날짜에 대해 티커 리스트 가져오기
current_date = datetime.datetime.strptime(start_date, '%Y%m%d')
while current_date.strftime('%Y%m%d') <= end_date:
    date_str = current_date.strftime('%Y%m%d')
    try:
        tickers_kosdaq = stock.get_market_ticker_list(date_str, market='KOSDAQ')
        tickers_kospi = stock.get_market_ticker_list(date_str, market='KOSPI')
        all_tickers.update(tickers_kosdaq)
        all_tickers.update(tickers_kospi)
        print(len(all_tickers), f'{date_str}')
        time.sleep(0.8)
    except Exception as e:
        print(f"Error on date {date_str}: {e}")
    current_date += datetime.timedelta(days=180)  # 180일 단위로 진행

conn = pymysql.connect(host=host, user=username, password=password, db=database, charset='utf8')
cur = conn.cursor()

# 티커 목록을 MySQL에 저장
for ticker in all_tickers:
    market = 'KOSDAQ' if ticker in tickers_kosdaq else 'KOSPI'
    name = stock.get_market_ticker_name(ticker)
    try:
        with conn.cursor() as cur:
            cur.execute('''
            INSERT INTO ticker_list (ticker, market, name) VALUES (%s, %s, %s)
            ON DUPLICATE KEY UPDATE market=VALUES(market), name=VALUES(name)
            ''', (ticker, market, name))
            
            # cur.execute('''
            # INSERT INTO ticker_status (ticker, status) VALUES (%s, 'pending')
            # ON DUPLICATE KEY UPDATE status=VALUES(status)
            # ''', (ticker,))
            conn.commit()
    except pymysql.InterfaceError as e:
        print(f"InterfaceError: {e}, reconnecting...")
        conn = pymysql.connect(host=host, user=username, password=password, db=database, charset='utf8') # 재연결 시도
    except pymysql.DatabaseError as e:
        print(f"DatabaseError: {e}")
        conn.rollback()  # 롤백하여 데이터 일관성 유지

conn.close()
print("모든 티커 목록이 성공적으로 저장되었습니다.")

0 19950102
956 19950701
1132 19951228
1190 19960625
1758 19961222
1827 19970620
1906 19971217
1930 19980615
1941 19981212
2003 19990610
2117 19991207
2298 20000604
2440 20001201
2525 20010530
2638 20011126
2773 20020525
2848 20021121
2895 20030520
2944 20031116
2983 20040514


KeyboardInterrupt: 

## Step 2: 데이터 갱신 및 새로운 티커 추가 (중단 후 재시작 가능)
* 한번 실행한 경우 몇 거래일 뒤에 다시 실행해야 오류가 발생하지 않음


In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
from pykrx import stock
import datetime
import os
# from sqlalchemy import create_engine
import pymysql
import numpy as np
import configparser

import os
import random

# 대기 시간 설정 함수
def random_sleep(min_seconds=0.5, max_seconds=1.5):
    time.sleep(random.uniform(min_seconds, max_seconds))
    # ATR 계산 함수 추가
def calculate_atr(df, period=14):
    high = df['high']
    low = df['low']
    close = df['close']
    
    # True Range (TR) 계산
    tr = np.maximum(
        high - low,
        np.abs(high - close.shift(1)),
        np.abs(low - close.shift(1))
    )
    
    # ATR 계산
    atr = tr.rolling(window=period).mean()
    return atr

# 정규화된 ATR 계산 함수
def calculate_normalized_atr(df, period=14):
    atr = calculate_atr(df, period)
    return (atr / df['close']) * 100
"""
# 설정 파일 읽기
config = configparser.ConfigParser()
config.read('config.ini')

username = config['mysql']['username']
password = config['mysql']['password']
host = config['mysql']['host']
database = config['mysql']['database']
"""


# ConfigParser 객체 생성
config = configparser.ConfigParser()

# config.ini 파일 경로 설정
config_file_path = 'E:\\AI\\pythonProject\\venv\\masicsplit\\config.ini'

# 경로가 올바른지 확인
if os.path.exists(config_file_path):
    print("config.ini 파일을 찾았습니다.")
    config.read(config_file_path)
else:
    print("config.ini 파일을 찾을 수 없습니다. 경로를 확인하세요.")

# mysql 섹션에서 설정값 가져오기
try:
    username = config['mysql']['user']
    password = config['mysql']['password']
    host = config['mysql']['host']
    database=config['mysql']['database']
except KeyError as e:
    print(f"설정 파일에서 키를 찾을 수 없습니다: {e}")
except Exception as e:
    print(f"기타 오류 발생: {e}")
    
    
conn = pymysql.connect(host=host, user=username, password=password, db=database, charset='utf8')

cur = conn.cursor()

# 조건 설정
per_threshold = 100
pbr_threshold = 10
div_threshold = 0  # 기준금리 참조

# 현재 날짜 기준으로 최근 월 종목 리스트 수집
current_date = datetime.datetime.now()
last_month_str = current_date.strftime("%Y%m") + '01'

# 데이터베이스에서 모든 티커 가져오기
cur.execute("SELECT ticker FROM ticker_list")
all_tickers = [row[0] for row in cur.fetchall()]

# 각 티커에 대한 처리
for ticker in all_tickers:
    # 이미 처리된 티커는 스킵
    cur.execute(f"SELECT status FROM ticker_status WHERE ticker = '{ticker}'")
    status = cur.fetchone()
    if status and status[0] == 'completed':
        continue
    
     # 티커의 끝자리가 0이 아닌 경우 스킵
    if ticker[-1] != '0':
        cur.execute(f"UPDATE ticker_status SET status = 'completed' WHERE ticker = '{ticker}'")
        conn.commit()
        print(f'{ticker} skipped because it does not end with 0')
        continue 
       
   
    # 데이터베이스에서 마지막 날짜 확인
    cur.execute(f"SELECT MAX(date) FROM stock_data WHERE ticker = '{ticker}'")
    last_date = cur.fetchone()[0]
    # 마지막 날짜가 현재 날짜와 30일 이하로 차이나는 경우 continue
    if last_date: # 이미 저장된 데이터가 있다면
        last_recorded_date = pd.to_datetime(last_date)
        if (current_date - last_recorded_date).days <= 2:
            print(f'{ticker} already done')
            continue
    random_sleep()
    name = stock.get_market_ticker_name(ticker)
    if "스팩" in name:
        cur.execute(f"UPDATE ticker_status SET status = 'completed' WHERE ticker = '{ticker}'")
        conn.commit()
        print(f'{ticker} skipped because it called {name}')
        continue
    # 새 데이터를 가져옴
    if last_date:
        start_date = (last_date + datetime.timedelta(days=1)).strftime("%Y%m%d")
    else:
        start_date = "19800102"
    
    end_date = current_date.strftime("%Y%m%d")
    
    random_sleep()
    df2 = stock.get_market_fundamental(start_date, end_date, ticker)
    if df2.empty or 'PER' not in df2.columns:
    
        cur.execute(f"UPDATE ticker_status SET status = 'completed' WHERE ticker = '{ticker}'")
        conn.commit()
    
        print(f'{ticker} no.2 pass condition')
        random_sleep()
        continue
    '''
    # 조건 확인
    if ('PER' in df2.columns and 'PBR' in df2.columns and 'DIV' in df2.columns):
        condition = (df2['PER'] > 0) & (df2['PER'] <= per_threshold) & \
                    (df2['PBR'] > 0) & (df2['PBR'] <= pbr_threshold) & \
                    (df2['DIV'] >= div_threshold)
        if not condition.any() and last_date is None:  #조건을 만족하지 않거나 새로운데이터가 없으면 해당 종목은 종료(completed)
            cur.execute(f"UPDATE ticker_status SET status = 'completed' WHERE ticker = '{ticker}'")
            conn.commit()
            print(f'{ticker} no.3 pass condition')
            time.sleep(5.5)  
            continue  # 조건을 만족하지 않으면 다음 티커로 넘어감
        
    else:
        cur.execute(f"UPDATE ticker_status SET status = 'completed' WHERE ticker = '{ticker}'")
        conn.commit()
        print(f'{ticker} no.4 pass condition')
        continue  # 필요한 컬럼이 없으면 넘어감
    '''        
    random_sleep()
    df1 = stock.get_market_ohlcv(start_date, end_date, ticker)
    

    if df1.empty:
        cur.execute(f"UPDATE ticker_status SET status = 'completed' WHERE ticker = '{ticker}'")
        conn.commit()
        print(f'{ticker} no.5 pass condition')
        random_sleep()
        continue

    # '고가' 컬럼이 있는지 확인
    if '고가' not in df1.columns:
        cur.execute(f"UPDATE ticker_status SET status = 'completed' WHERE ticker = '{ticker}'")
        conn.commit()
        print(f'{ticker} no.6 pass condition')
        random_sleep()
        continue
    random_sleep()
    
    # ATR 계산 및 정규화된 ATR 계산 추가
    df1['normalized_atr'] = calculate_normalized_atr(df1)
    df3 = stock.get_market_cap(start_date, end_date, ticker)
    
    
    df1.reset_index(inplace=True)
    df2.reset_index(inplace=True)
    df3.reset_index(inplace=True)
    # 컬럼명 통일
    df1.rename(columns={
        '날짜': 'date',
        '시가': 'open',
        '고가': 'high',
        '저가': 'low',
        '종가': 'close',
        '거래량': 'volume',
        
    }, inplace=True)

    df2.rename(columns={
        '날짜': 'date',
        'PER': 'PER',
        'PBR': 'PBR',
        'DIV': 'dividend',
        'BPS': 'BPS',
        'EPS': 'EPS',
        'DPS': 'DPS'
    }, inplace=True)

    df3.rename(columns={
        '날짜': 'date',
        '시가총액': 'market_cap',
        '상장주식수': 'shares_outstanding',
        '거래대금': 'value',
        '거래량': 'volume'
        
    }, inplace=True)

    
     # 날짜 기준으로 결합하고 중복된 열 제거
    merged_df = pd.merge(df1, df2, on='date', how='left', suffixes=('', '_duplicate'))
    merged_df = pd.merge(merged_df, df3, on='date', how='left', suffixes=('', '_duplicate'))
    
    # 중복된 열 제거
    for column in merged_df.columns:
        if 'duplicate' in column:
            base_column = column.replace('_duplicate', '')
            if base_column in merged_df.columns:
                merged_df.drop(column, axis=1, inplace=True)
    
    # 필요한 열만 남기기
    merged_df = merged_df[['date', 'open', 'high', 'low', 'close', 'volume', 'value', 'market_cap', 'shares_outstanding', 'PER', 'PBR', 'dividend', 'BPS', 'EPS', 'DPS','normalized_atr']]
    # merged_df 상태 출력
  
    # NaN 값을 None으로 변환 (개별적으로 처리)
    merged_df = merged_df.replace({np.nan: None})
    # # merged_df 상태 출력
    # print("merged_df:")
    # print(merged_df.head())

    merged_df['ticker'] = ticker
    merged_df['name'] = name
  
    

    # 기존 데이터를 가져와서 새로운 데이터와 결합
    if last_date:
        cur.execute(f"SELECT * FROM stock_data WHERE ticker = '{ticker}' AND date <= '{last_date}'")
        existing_data = pd.DataFrame(cur.fetchall(), columns=['ticker', 'name', 'date', 'open', 'high', 'low', 'close', 'volume', 'value', 'market_cap', 'shares_outstanding', 'PER', 'PBR', 'dividend', 'BPS', 'EPS', 'DPS','normalized_value'])
        existing_data['date'] = pd.to_datetime(existing_data['date'])
        existing_data.set_index('date', inplace=True)
        merged_df.set_index('date', inplace=True)
        combined_df = pd.concat([existing_data, merged_df], axis=0, join='outer')
          # 여기서 결합 전에 각 데이터프레임 확인

        
    else: # 기존데이터가 없으면 로드된 세 DF만 결합
        combined_df = merged_df
    # print(combined_df)
        
        
    import pandas as pd
    import numpy as np

    combined_df['min_close'] = combined_df['close'].rolling(window=252*5, min_periods=252).min()
    combined_df['max_close'] = combined_df['close'].rolling(window=252*5, min_periods=252).max()

    # 각 행에 대해 'min_close'와 'max_close'가 0이 아닌지 확인하여 'normalized_value' 계산
    def calculate_normalized_value(row):
        if pd.notnull(row['min_close']) and pd.notnull(row['max_close']) and (row['max_close'] - row['min_close']) != 0:
            return (row['close'] - row['min_close']) / (row['max_close'] - row['min_close']) * 100
        else:
            return None
    try:
        combined_df['normalized_value'] = combined_df.apply(calculate_normalized_value, axis=1)
    except TypeError as e:
        print(f"TypeError encountered: {e}")
        
        print(f'{ticker} no.7 pass condition')
        continue

    if combined_df['normalized_value'].isnull().all():
        cur.execute(f"UPDATE ticker_status SET status = 'completed' WHERE ticker = '{ticker}'")
        conn.commit()
        print(f'{ticker} no.8 pass condition')
        continue


    combined_df.reset_index(inplace=True)
    
    combined_df = combined_df.replace({np.nan: None})
    

    # 데이터베이스에 저장
    for _, row in combined_df.iterrows():
        sql = '''
        INSERT INTO stock_data (ticker, name, date, open, high, low, close, volume, value, market_cap, shares_outstanding, PER, PBR, dividend, BPS, EPS, DPS, normalized_value, normalized_atr)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE
            open=VALUES(open),
            high=VALUES(high),
            low=VALUES(low),
            close=VALUES(close),
            volume=VALUES(volume),
            value=VALUES(value),
            market_cap=VALUES(market_cap),
            shares_outstanding=VALUES(shares_outstanding),
            PER=VALUES(PER),
            PBR=VALUES(PBR),
            dividend=VALUES(dividend),
            BPS=VALUES(BPS),
            EPS=VALUES(EPS),
            DPS=VALUES(DPS),
            normalized_value=VALUES(normalized_value),
            normalized_atr=VALUES(normalized_atr)
        '''
        cur.execute(sql, (
            row['ticker'],
            row['name'],
            row['date'].strftime('%Y-%m-%d') if row['date'] else None,
            row['open'],
            row['high'],
            row['low'],
            row['close'],
            int(row['volume']) if row['volume'] is not None else None,
            int(row['value']) if row['value'] is not None else None,
            int(row['market_cap']) if row['market_cap'] is not None else None,
            int(row['shares_outstanding']) if row['shares_outstanding'] is not None else None,
            row['PER'],
            row['PBR'],
            row['dividend'],
            row['BPS'],
            row['EPS'],
            row['DPS'],
            row['normalized_value'],
            row['normalized_atr']
        ))

    conn.commit()
     # 최근 60일 동안 데이터가 있는지 확인
    last_recorded_date = pd.to_datetime(combined_df['date']).max()
    if (current_date - last_recorded_date).days > 60:
        print(current_date - last_recorded_date,current_date,last_recorded_date)
        cur.execute(f"UPDATE ticker_status SET status = 'completed' WHERE ticker = '{ticker}'")
        conn.commit()
    print(f"{ticker} 데이터베이스에 저장 완료")
    random_sleep()  


cur.close()
conn.close()
print("모든 데이터가 성공적으로 저장되었습니다.")

config.ini 파일을 찾았습니다.
000010 no.5 pass condition
000020 데이터베이스에 저장 완료
000021 skipped because it does not end with 0
000030 no.2 pass condition
000040 already done
000041 skipped because it does not end with 0
000042 skipped because it does not end with 0
000050 already done
000060 no.2 pass condition
000070 already done
000071 skipped because it does not end with 0
000075 skipped because it does not end with 0
000076 skipped because it does not end with 0
000080 already done
000081 skipped because it does not end with 0
000085 skipped because it does not end with 0
000086 skipped because it does not end with 0
000087 skipped because it does not end with 0
000090 no.5 pass condition
000091 skipped because it does not end with 0
000100 already done
000101 skipped because it does not end with 0
000102 skipped because it does not end with 0
000105 skipped because it does not end with 0
000106 skipped because it does not end with 0
000107 skipped because it does not end with 0
000110 no.5 p

# normalized_atr을 계산 후 테이블에 삽입

In [7]:
import pymysql
import pandas as pd
import numpy as np
import configparser
import os

# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.chrome.service import Service
# from webdriver_manager.chrome import ChromeDriverManager




# ATR 계산 함수
def calculate_atr(df, period=14):
    high = df['high']
    low = df['low']
    close = df['close']
    
    # True Range(TR)을 계산
    tr = np.maximum(
        high - low,
        np.abs(high - close.shift(1)),
        np.abs(low - close.shift(1))
    )
    
    # ATR 계산: TR의 이동 평균
    atr = tr.rolling(window=period).mean()
    return atr

# 정규화된 ATR 계산 함수
def calculate_normalized_atr(df, period=14):
    atr = calculate_atr(df, period)
    return (atr / df['close']) * 100

# ConfigParser 객체 생성
config = configparser.ConfigParser()

# config.ini 파일 경로 설정
config_file_path = 'E:\\AI\\pythonProject\\venv\\masicsplit\\config.ini'

# 경로가 올바른지 확인
if os.path.exists(config_file_path):
    print("config.ini 파일을 찾았습니다.")
    config.read(config_file_path)
else:
    print("config.ini 파일을 찾을 수 없습니다. 경로를 확인하세요.")

# mysql 섹션에서 설정값 가져오기
try:
    username = config['mysql']['user']
    password = config['mysql']['password']
    host = config['mysql']['host']
    database=config['mysql']['database']
except KeyError as e:
    print(f"설정 파일에서 키를 찾을 수 없습니다: {e}")
except Exception as e:
    print(f"기타 오류 발생: {e}")
    
# 데이터베이스에 연결    
conn = pymysql.connect(host=host, user=username, password=password, db=database, charset='utf8')

cur = conn.cursor()

# 데이터베이스에서 모든 티커 가져오기
cur.execute("SELECT DISTINCT ticker FROM stock_data")
tickers = [row[0] for row in cur.fetchall()]

# 각 티커에 대해 데이터 처리
for ticker in tickers:
    print(f"Processing ticker: {ticker}")
    
    # 해당 티커의 데이터를 불러오기
    query = f"SELECT date, open, high, low, close FROM stock_data WHERE ticker = '{ticker}'"
    cur.execute(query)
    rows = cur.fetchall()
    
    # 데이터가 없는 경우 스킵
    if len(rows) == 0:
        continue
    
    # 데이터를 DataFrame으로 변환
    df = pd.DataFrame(rows, columns=['date', 'open', 'high', 'low', 'close'])
    
    # ATR 및 정규화된 ATR 값 계산
    df['normalized_atr'] = calculate_normalized_atr(df)
    # 'NaN' 값을 'None'으로 변환하여 MySQL에서 NULL로 처리되도록 함
    df['normalized_atr'] = df['normalized_atr'].replace({np.nan: None})
    
    # MySQL 데이터베이스에 업데이트
    for index, row in df.iterrows():
        update_query = '''
        UPDATE stock_data
        SET normalized_atr = %s
        WHERE ticker = %s AND date = %s
        '''
        cur.execute(update_query, (row['normalized_atr'], ticker, row['date']))
        conn.commit()

    print(f"Ticker {ticker} processed successfully.")
# 커넥션 종료
cur.close()
conn.close()

print("All tickers processed.")

config.ini 파일을 찾았습니다.
Processing ticker: 000020
Ticker 000020 processed successfully.
Processing ticker: 000030
Ticker 000030 processed successfully.
Processing ticker: 000040
Ticker 000040 processed successfully.
Processing ticker: 000050
Ticker 000050 processed successfully.
Processing ticker: 000060
Ticker 000060 processed successfully.
Processing ticker: 000070
Ticker 000070 processed successfully.
Processing ticker: 000080
Ticker 000080 processed successfully.
Processing ticker: 000100
Ticker 000100 processed successfully.
Processing ticker: 000120
Ticker 000120 processed successfully.
Processing ticker: 000140
Ticker 000140 processed successfully.
Processing ticker: 000150
Ticker 000150 processed successfully.
Processing ticker: 000180
Ticker 000180 processed successfully.
Processing ticker: 000210
Ticker 000210 processed successfully.
Processing ticker: 000220
Ticker 000220 processed successfully.
Processing ticker: 000230
Ticker 000230 processed successfully.
Processing ticker:

## ETF 이름 & 티커 리스트 생성  

In [6]:
# 크롬 드라이버 설정
options = webdriver.ChromeOptions()
options.add_argument('headless')  # 브라우저 창을 띄우지 않음
options.add_argument('disable-gpu')  # GPU 가속 비활성화
options.add_argument('lang=ko_KR')  # 한국어 페이지

# 크롬 드라이버 초기화
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

In [6]:
import pandas as pd
import pymysql
import configparser
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time

# 설정 파일 읽기
config = configparser.ConfigParser()
config.read('config.ini')

username = config['mysql']['username']
password = config['mysql']['password']
host = config['mysql']['host']
database = config['mysql']['database']

# MySQL 연결
conn = pymysql.connect(host=host, user=username, password=password, db=database, charset='utf8')
cur = conn.cursor()

# 크롬 드라이버 설정
options = webdriver.ChromeOptions()
options.add_argument('headless')  # 브라우저 창을 띄우지 않음
options.add_argument('disable-gpu')  # GPU 가속 비활성화
options.add_argument('lang=ko_KR')  # 한국어 페이지

# 크롬 드라이버 초기화
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
# URL에서 HTML 컨텐츠를 가져옴
url = 'https://finance.naver.com/sise/etf.naver'
driver.get(url)

# 종목명과 티커를 추출하여 리스트로 저장
etf_data = []
rows = driver.find_elements(By.CSS_SELECTOR, '#etfItemTable > tr')
for row in rows:
    name_col = row.find_elements(By.CSS_SELECTOR, 'td.ctg a')
    if name_col:
        name = name_col[0].text.strip()
        ticker = name_col[0].get_attribute('href').split('=')[-1]  # 종목명 링크에서 티커 추출
        etf_data.append([name, ticker])

# 드라이버 종료
driver.quit()

# DataFrame 생성
df = pd.DataFrame(etf_data, columns=['종목명', '티커'])

# DataFrame을 Excel 파일로 저장
excel_filename = 'etf_list.xlsx'
df.to_excel(excel_filename, index=False)
print(f"Excel 파일로 저장되었습니다: {excel_filename}")

# DataFrame을 MySQL에 저장
create_table_query = '''
CREATE TABLE IF NOT EXISTS etf_list (
    name VARCHAR(100),
    ticker VARCHAR(20),
    PRIMARY KEY (ticker)
)
'''
cur.execute(create_table_query)
conn.commit()

# MySQL에 데이터 삽입
for index, row in df.iterrows():
    insert_query = '''
    INSERT INTO etf_list (name, ticker) VALUES (%s, %s)
    ON DUPLICATE KEY UPDATE name=VALUES(name)
    '''
    cur.execute(insert_query, (row['종목명'], row['티커']))

conn.commit()
cur.close()
conn.close()
print("MySQL 데이터베이스에 저장되었습니다.")

Excel 파일로 저장되었습니다: etf_list.xlsx
MySQL 데이터베이스에 저장되었습니다.
